In [13]:
import json
import pandas as pd
import geopandas as gpd
import os

/Users/james/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-13-47c6e33d18f2>:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS dire

In [5]:
def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories

base_dir = 'turkey_earthquake/'
label_base_folder = base_dir + 'labels/copernicus_earthquake_turkey_json_02-23/'
city_list = list_subdirectories(label_base_folder)
city_list

[]

In [8]:
base_dir = 'turkey_earthquake/'
label_dir = base_dir + 'labels/copernicus_earthquake_turkey_json_02-23/'
filenames = [f for f in os.listdir(label_dir) if ".json" in f]
filenames[:5]

['EMSR648_AOI15_GRA_MONIT01_builtUpP_r1_v1.json',
 'EMSR648_AOI09_GRA_MONIT01_builtUpA_r1_v1.json',
 'EMSR648_AOI16_GRA_MONIT01_builtUpA_r1_v1.json',
 'EMSR648_AOI10_GRA_MONIT02_builtUpA_r1_v1.json',
 'EMSR648_AOI05_GRA_PRODUCT_builtUpA_r1_v1.json']

In [29]:
# Read the GeoJSON file into a GeoPandas DataFrame
f = filenames[0]
json_file_path = label_dir + f
gdf = gpd.read_file(json_file_path)
gdf.head()

,obj_type,name,info,damage_gra,det_method,notation,or_src_id,dmg_src_id,cd_value,geometry
0,12-Non-residential Buildings,Bahçe,"1241-Communication buildings, stations, termin...",No visible damage,Photo-interpretation,Building point,994,2,Not Applicable,POINT (36.55905 37.18686)
1,12-Non-residential Buildings,Unknown,1272-Buildings used as places of worship and f...,No visible damage,Photo-interpretation,Building point,994,2,Not Applicable,POINT (36.57082 37.19470)
2,12-Non-residential Buildings,Unknown,1272-Buildings used as places of worship and f...,No visible damage,Photo-interpretation,Building point,994,2,Not Applicable,POINT (36.57653 37.19668)
3,12-Non-residential Buildings,Unknown,1272-Buildings used as places of worship and f...,No visible damage,Photo-interpretation,Building point,994,2,Not Applicable,POINT (36.56481 37.19341)
4,12-Non-residential Buildings,Unknown,1272-Buildings used as places of worship and f...,No visible damage,Photo-interpretation,Building point,994,2,Not Applicable,POINT (36.57323 37.20474)


In [40]:
count_df = gdf.groupby('damage_gra').count()[['name']].reset_index()
count_df['AOIno'] = int(f.split('_')[1][3:])
count_df = count_df.rename({'name':'count'},axis=1)
count_df

,damage_gra,count,AOIno
0,Damaged,2,15
1,No visible damage,3217,15


In [43]:
def get_dmg_df(f):
    json_file_path = label_dir + f
    gdf = gpd.read_file(json_file_path)
    count_df = gdf.groupby('damage_gra').count()[['name']].reset_index()
    count_df['AOIno'] = int(f.split('_')[1][3:])
    count_df = count_df.rename({'name':'count'},axis=1)
    return count_df

# get summary stat df
dmg_df_list = []
for f in filenames:
    dmg_df_list.append(get_dmg_df(f))
dmg_df = pd.concat(dmg_df_list,sort=False)
dmg_df.head()

,damage_gra,count,AOIno
0,Damaged,2,15
1,No visible damage,3217,15
0,Damaged,14,9
1,No visible damage,331,9
0,Damaged,41,16


In [52]:
# read metadata df
metadata_df = pd.read_csv(base_dir+'labels/turkey_earthquake_copernicus_metadata.csv')
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AOIno                 20 non-null     int64  
 1   AOIname               20 non-null     object 
 2   Area(km2)             20 non-null     int64  
 3   Estimated population  20 non-null     int64  
 4   Production status     20 non-null     object 
 5   Affected roads (km)   14 non-null     float64
 6   Affected buildings    20 non-null     int64  
 7   Label Format          20 non-null     object 
 8   Filename              20 non-null     object 
dtypes: float64(1), int64(4), object(4)
memory usage: 1.5+ KB


In [59]:
dmg_pivot_df = dmg_df.pivot_table(columns=['damage_gra'],index=['AOIno'],values=['count']).fillna(0).reset_index()
dmg_pivot_df.columns = ['AOIno','NumDamaged','NumDestroyed','NumNoVisibleDamage','NumPossiblyDamaged']
idx = 'AOIno'
dmg_pivot_df = dmg_pivot_df.set_index(idx).join(metadata_df[['AOIno','AOIname','Estimated population']].set_index(idx),how='left').reset_index()
dmg_pivot_df = dmg_pivot_df[['AOIno','AOIname','NumNoVisibleDamage','NumPossiblyDamaged','NumDamaged','NumDestroyed','Estimated population']]
dmg_pivot_df['NumBuildings'] = dmg_pivot_df['NumNoVisibleDamage'] + dmg_pivot_df['NumPossiblyDamaged'] + dmg_pivot_df['NumDamaged'] + dmg_pivot_df['NumDestroyed'] 
dmg_pivot_df['NumAffectedBuildings'] = dmg_pivot_df['NumPossiblyDamaged'] + dmg_pivot_df['NumDamaged'] + dmg_pivot_df['NumDestroyed']
dmg_pivot_df['PercentAffectedBuildings'] = (dmg_pivot_df['NumAffectedBuildings'] / dmg_pivot_df['NumBuildings']*100).round(1)
dmg_pivot_df

,AOIno,AOIname,NumNoVisibleDamage,NumPossiblyDamaged,NumDamaged,NumDestroyed,Estimated population,NumBuildings,NumAffectedBuildings,PercentAffectedBuildings
0,1,Gaziantep,10888.0,19.0,9.0,44.0,1191034,10960.0,72.0,0.7
1,2,Adiyaman,2900.0,17.0,87.0,32.0,209256,3036.0,136.0,4.5
2,3,Diyarbakir,0.0,58.0,24.0,15.0,562481,97.0,97.0,100.0
3,4,Kahramanmaras,3692.0,456.0,185.0,286.0,384404,4619.0,927.0,20.1
4,5,Malatya,3267.0,93.0,157.0,102.0,423295,3619.0,352.0,9.7
5,6,Osmaniye,2075.0,83.0,133.0,33.0,231122,2324.0,249.0,10.7
6,7,Sanliurfa,6068.0,3.0,2.0,0.0,501655,6073.0,5.0,0.1
7,8,Pazarcik,15.0,68.0,1.0,10.0,25477,94.0,79.0,84.0
8,9,Cumhuriyet,331.0,0.0,14.0,0.0,16653,345.0,14.0,4.1
9,10,Islahiye,478.0,33.0,69.0,65.0,16036,645.0,167.0,25.9


In [60]:
dmg_pivot_df.to_csv(base_dir+'labels/turkey_earthquake_copernicus_stats.csv',index=False)